In [30]:
import re
import json
import itertools
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.utils.data as data

from tqdm import tqdm
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from transformers import AlbertTokenizer
from setup import process_file
from models import investorConferenceAnalyzer
from utils import Treebank, collate_fn, nll_loss_label_smoothing

def get_args():
    def args():
        pass
    args.train_data_path = './data/trees/train.txt'
    args.dev_data_path = './data/trees/dev.txt'
    args.test_data_path = './data/trees/test.txt'
    args.pce_model = 'albert-base-v2'
    args.train_record_file = './data/train.npz'
    args.num_labels = 5
    args.batch_size = 4
    args.train_eval_file = './data/train_eval.json'
    args.tiny_eval_file = './data/train_tiny_eval.json'
    args.train_meta_file = './data/train_meta.json'
    args.train_record_file = './data/train.npz'
    args.train_eval_file = './data/train_eval.json'
    return args

args = get_args()
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[autoreload of utils failed: Traceback (most recent call last):
  File "/Users/staff/Desktop/.pyenv/versions/investor_conference/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/staff/Desktop/.pyenv/versions/investor_conference/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/staff/Desktop/.pyenv/versions/3.8.0/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/staff/Desktop/.pyenv/versions/3.8.0/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 779, in exec_module
  File "<frozen importlib._bootstrap_external>", line 916, in get_co

In [32]:
help(nll_loss_label_smoothing)

Help on function nll_loss_label_smoothing in module utils:

nll_loss_label_smoothing(log_p, targets, eps=0.1)
    Calculate cross entropy loss with label smoothing
    
    Args:
        log_p (torch.Tensor): Log probability.
        targets (torch.Tensor): Target labels.
        eps (float): Label smoothing parameter.
    
    Returns:
        loss[torch.Tensor]: Mean entropy loss with label smoothing.



In [28]:
changed_files = ('record_file', 'eval_file', 'meta_file')
for key, value in vars(args).items():
    if any(name in key for name in changed_files):
        value = vars(args)[key]
        path = value.split(".")
        path[-2] = f'{path[-2]}_b'
        value = '.'.join(path)
        setattr(args, key, value)

In [4]:
model = investorConferenceAnalyzer(args.pce_model, args.num_labels)
dataset = Treebank(args.train_record_file)
dataloader = data.DataLoader(dataset,
                             batch_size=args.batch_size,
                             shuffle=False,
                             num_workers=4,
                             collate_fn=collate_fn)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

In [43]:
input_idxs, token_type_idxs, attention_masks, ys, ids = dataset[:2]
log_p = model(input_idxs, token_type_idxs, attention_masks)
nll_loss = -log_p.gather(dim=-1, index=ys.unsqueeze(1)).squeeze(-1)
smooth_loss = -log_p.mean(dim=-1)
loss = nll_loss * (1 - eps) + smooth_loss * eps
loss.mean()

tensor([1.5523, 1.8172], grad_fn=<AddBackward0>)